In [1]:
import os
import geopandas as gpd
import pandas as pd

import shapely.geometry as geom

In [2]:
vector_dir = '../Mask_RCNN/datasets/canopy/real_test/images/TIF/test_2/results'
# vector_dir = '../Mask_RCNN/datasets/canopy/real_test/images_2/results'

result_paths = []
for filename in os.listdir(vector_dir):
    if os.path.splitext(filename)[1].lower() in ['.geojson']:
        result_paths.append(os.path.join(vector_dir, filename))

In [ ]:
len(result_paths)

In [7]:
pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols

,ID,geometry
0,10.0,"POLYGON ((294097.584 7516109.448, 294097.584 7..."
1,13.0,"POLYGON ((294093.534 7516108.798, 294093.534 7..."
2,8.0,"POLYGON ((294100.483 7516109.448, 294100.483 7..."
3,3.0,"POLYGON ((294104.183 7516109.248, 294104.183 7..."
4,12.0,"POLYGON ((294093.284 7516105.948, 294093.284 7..."
...,...,...
1062,6.0,"POLYGON ((294051.687 7516022.458, 294051.687 7..."
1063,8.0,"POLYGON ((294053.987 7516021.008, 294053.987 7..."
1064,2.0,"POLYGON ((294056.986 7516020.708, 294056.986 7..."
1065,9.0,"POLYGON ((294047.387 7516016.459, 294047.387 7..."


In [9]:
for i in result_paths:
    
    if i == result_paths[0]:
        pols = gpd.read_file(i)
    
    df_aux = gpd.read_file(i)
    
#     print(df_aux)
    
    for f in df_aux.index:
        
        if i == result_paths[0]:
            pass
        else:
            pols = pd.concat([pols, df_aux.loc[[f]]], axis=0)

pols.reset_index(drop=True, inplace=True)
pols['ID'] = pols.index
pols.insert(1,column='union',value=0)
pols

,ID,union,geometry
0,0,0,"POLYGON ((294097.584 7516109.448, 294097.584 7..."
1,1,0,"POLYGON ((294093.534 7516108.798, 294093.534 7..."
2,2,0,"POLYGON ((294100.483 7516109.448, 294100.483 7..."
3,3,0,"POLYGON ((294104.183 7516109.248, 294104.183 7..."
4,4,0,"POLYGON ((294093.284 7516105.948, 294093.284 7..."
...,...,...,...
1062,1062,0,"POLYGON ((294051.687 7516022.458, 294051.687 7..."
1063,1063,0,"POLYGON ((294053.987 7516021.008, 294053.987 7..."
1064,1064,0,"POLYGON ((294056.986 7516020.708, 294056.986 7..."
1065,1065,0,"POLYGON ((294047.387 7516016.459, 294047.387 7..."


In [10]:
delete_lines = []
for i in pols.index:
    
    if pols.loc[i,'union'] == 0:
    
        polygon = pols.loc[i,'geometry']

        intersec = pols.overlaps(polygon)

        intersec = intersec[intersec]

        list_result = list(intersec.index)
        
        if len(list_result) != 0:
            maior = pols.loc[list_result,'geometry'].area.idxmax(axis=0)
        
            for z in list_result:
                inter_poly = pols.loc[z,'geometry'].intersection(pols.loc[maior,'geometry'])
                if inter_poly.geom_type == 'Polygon' and inter_poly.area > 0.1:
                    delete_lines.append(z)
                    pols.at[z,'union'] = i
    
    if pols.loc[i,'union'] == 0:
        pols.at[i,'union'] = i

# pols.drop(labels=delete_lines, axis=0, inplace=True)
        
pols

,ID,union,geometry
0,0,0,"POLYGON ((294097.584 7516109.448, 294097.584 7..."
1,1,1,"POLYGON ((294093.534 7516108.798, 294093.534 7..."
2,2,2,"POLYGON ((294100.483 7516109.448, 294100.483 7..."
3,3,108,"POLYGON ((294104.183 7516109.248, 294104.183 7..."
4,4,4,"POLYGON ((294093.284 7516105.948, 294093.284 7..."
...,...,...,...
1062,1062,1062,"POLYGON ((294051.687 7516022.458, 294051.687 7..."
1063,1063,1063,"POLYGON ((294053.987 7516021.008, 294053.987 7..."
1064,1064,1064,"POLYGON ((294056.986 7516020.708, 294056.986 7..."
1065,1065,146,"POLYGON ((294047.387 7516016.459, 294047.387 7..."


In [11]:
pols.dissolve(by='union')

,geometry,ID
union,,
0,"POLYGON ((294097.584 7516109.448, 294097.584 7...",0
1,"POLYGON ((294093.534 7516108.798, 294093.534 7...",1
2,"POLYGON ((294100.883 7516109.448, 294102.883 7...",2
3,"POLYGON ((294102.033 7516110.698, 294102.033 7...",905
4,"POLYGON ((294093.384 7516104.498, 294093.384 7...",4
...,...,...
1061,"POLYGON ((294049.637 7516023.658, 294049.637 7...",1061
1062,"POLYGON ((294051.687 7516022.458, 294051.687 7...",1062
1063,"POLYGON ((294053.987 7516021.008, 294053.987 7...",1063


In [ ]:
pols.to_file('../Mask_RCNN/datasets/canopy/test_union_2.geojson', driver='GeoJSON')

In [ ]:
pols.drop('union', axis=1,inplace=True)